In [1]:
import meshcat
from utils import *
from interface import ExoSkeletonUDPInterface
import time 
import collections


from estimation_problem import solve_estimation_problem
from utils import ArmMeasurementCurrent
from arm_model import create_arm
from reaching_problem import solve_reaching_problem
from pinocchio.visualize import MeshcatVisualizer as Visualizer
import pinocchio as pin



In [2]:
interface = ExoSkeletonUDPInterface()
counter = 0
interface.calibrate()

calibration ...
setting offset ...
finished calibration ...


In [9]:
rmodel, rdata, gmodel, cmodel = create_arm()
viz = Visualizer(rmodel, gmodel, cmodel)
viz.initViewer(open=True)
viz.loadViewerModel()
viz.initializeFrames()
viz.display_frames = True

add_frame("hand", viz.viewer)
add_frame("shoulder", viz.viewer)

# viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


In [11]:
names = ['hand', 'base', 'shoulder']
data_offset = []
offset = [pin.utils.rpyToMatrix(np.pi/2.0, 0, 0), 0]
counter = 0
T = 10
estimate_x0 = np.zeros(rmodel.nq + rmodel.nv)

measurement = collections.deque(maxlen=T)

target = np.array([ 0.40196694,  0.00887781, -0.0389792 ])
offset_shoulder = 0
torque = 0.0
while True:
    interface.setCommand([0], [0.], [0], [0], [torque])
    time.sleep(0.002)
    state = interface.getState()
    # print(state["base_acc"], state["shoulder_acc"], state["wrist_acc"])
    base = Rotation.from_quat(state["base_ori"][0]).as_matrix()
    shoulder = Rotation.from_quat(state["shoulder_ori"][0]).as_matrix()
    hand = Rotation.from_quat(state["wrist_ori"][0]).as_matrix()
    if counter < 100:
        data_offset.append([base.T @ shoulder, base.T @ hand])
    elif counter == 100:
        offset[0] = offset[0] @ data_offset[50][0].T
        offset[1] = data_offset[50][1].T
    else:
        update_frame('hand', viz.viewer, offset[1] @ base.T @ hand, [0.5, 0, 0])
        update_frame("shoulder", viz.viewer,   offset[0] @ base.T @ shoulder)
        measurement.append(ArmMeasurementCurrent( offset[0] @ base.T @ shoulder, offset[1] @ base.T @ hand))
        if counter > T + 100:
            st = time.time()
            estimate = solve_estimation_problem(measurement, T, rmodel, estimate_x0)
            q,v = estimate.xs[-1][:rmodel.nq], estimate.xs[-1][rmodel.nq:] 

            if counter > 100 and counter < 120:
                offset_shoulder = state["q"] + estimate.xs[-1][1] 
            else:
                estimate.xs[-1][1] += offset_shoulder
            
            viz.display(estimate.xs[-1][:rmodel.nq])
            # estimate_x0 = estimate.xs[-1]
            state["q"] -= offset_shoulder

            # if counter > 120:
            #     ddp = solve_reaching_problem(target, q, rmodel)
            #     for i in range(len(ddp.xs)):
            #         viz.display(ddp.xs[i][:rmodel.nq])
            #     torque = max(0,-0.2*ddp.us[0][1])
            #     print(torque,  estimate.xs[-1][1], state["q"], state["shoulder_acc"])


    counter += 1


please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
p

/tmp/ipykernel_276788/2161563290.py:38: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  estimate.xs[-1][1] += offset_shoulder


please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
please calibrate IMU to continue ...
p

KeyboardInterrupt: 